In [11]:
#import libraries
import pandas as pd

In [12]:
#read dataset
data = pd.read_csv("kpopdataset.csv",sep="\t",delimiter = ";")
data.head()

,rank,name,agency,real name,group,age,height,weight,blood type,origin,country,position,industry,gender,curr date,birth date
0,1.0,Jinah,주식회사 씨그널엔터테인먼트그룹,Seo Jin Ah,Berry Chu,37.0,NaN,NaN,NaN,Seoul,South Korea,"Leader, Vocalist",K-pop,Female,23.04.2020,29.04.1983
1,2.0,Hayul,주식회사 씨그널엔터테인먼트그룹,Kim Ha Yul,Berry Chu,33.0,NaN,NaN,NaN,Seoul,South Korea,Vocalist,K-pop,Female,23.04.2020,18.10.1986
2,3.0,Jieun,주식회사 씨그널엔터테인먼트그룹,Han Ji Eun,Berry Chu,33.0,NaN,NaN,NaN,Seoul,South Korea,Vocalist,K-pop,Female,23.04.2020,12.01.1987
3,4.0,Yunmi,주식회사 씨그널엔터테인먼트그룹,Heo Yun Mi,Berry Chu,32.0,NaN,NaN,NaN,Incheon,South Korea,Vocalist,K-pop,Female,23.04.2020,23.03.1988
4,5.0,Haum,주식회사 씨그널엔터테인먼트그룹,Kim Haum,Berry Chu,29.0,NaN,NaN,NaN,Seoul,South Korea,"Vocalist, Maknae",K-pop,Female,23.04.2020,20.10.1990


In [13]:
data.shape

(1902, 16)

In [14]:
#missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1902 entries, 0 to 1901
Data columns (total 16 columns):
rank          1901 non-null float64
name          1901 non-null object
agency        1874 non-null object
real name     1566 non-null object
group         1901 non-null object
age           1725 non-null float64
height        1603 non-null float64
weight        1253 non-null float64
blood type    1349 non-null object
origin        686 non-null object
country       1895 non-null object
position      1836 non-null object
industry      1901 non-null object
gender        1901 non-null object
curr date     1901 non-null object
birth date    1725 non-null object
dtypes: float64(4), object(12)
memory usage: 237.8+ KB


In [15]:
#drop the columns
data.drop('real name',axis=1,inplace=True)
data.drop('origin',axis=1,inplace=True)
data.drop('industry',axis=1,inplace=True)
data.drop('curr date',axis=1,inplace=True)
data.drop('birth date',axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1902 entries, 0 to 1901
Data columns (total 11 columns):
rank          1901 non-null float64
name          1901 non-null object
agency        1874 non-null object
group         1901 non-null object
age           1725 non-null float64
height        1603 non-null float64
weight        1253 non-null float64
blood type    1349 non-null object
country       1895 non-null object
position      1836 non-null object
gender        1901 non-null object
dtypes: float64(4), object(7)
memory usage: 163.5+ KB


In [23]:
data["age"].fillna(data.groupby("name")["age"].transform("median"), inplace=True)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1902 entries, 0 to 1901
Data columns (total 11 columns):
rank          1901 non-null float64
name          1901 non-null object
agency        1874 non-null object
group         1901 non-null object
age           1765 non-null float64
height        1603 non-null float64
weight        1253 non-null float64
blood type    1349 non-null object
country       1895 non-null object
position      1836 non-null object
gender        1901 non-null object
dtypes: float64(4), object(7)
memory usage: 163.5+ KB


In [24]:
data.head(66)

,rank,name,agency,group,age,height,weight,blood type,country,position,gender
0,1.0,Jinah,주식회사 씨그널엔터테인먼트그룹,Berry Chu,37.0,NaN,NaN,NaN,South Korea,"Leader, Vocalist",Female
1,2.0,Hayul,주식회사 씨그널엔터테인먼트그룹,Berry Chu,33.0,NaN,NaN,NaN,South Korea,Vocalist,Female
2,3.0,Jieun,주식회사 씨그널엔터테인먼트그룹,Berry Chu,33.0,NaN,NaN,NaN,South Korea,Vocalist,Female
3,4.0,Yunmi,주식회사 씨그널엔터테인먼트그룹,Berry Chu,32.0,NaN,NaN,NaN,South Korea,Vocalist,Female
4,5.0,Haum,주식회사 씨그널엔터테인먼트그룹,Berry Chu,29.0,NaN,NaN,NaN,South Korea,"Vocalist, Maknae",Female
5,6.0,Soyou,주식회사 씨그널엔터테인먼트그룹,Berry Chu,34.0,NaN,NaN,NaN,South Korea,Vocalist,Female
6,7.0,Vanya,Zenith Media Contents,Z-Girls,23.0,159.0,50.0,O,Indonesia,"Lead Vocalist, Main Dancer, Lead Rapper",Female
7,8.0,Roy,Zenith Media Contents,Z-Boys,23.0,NaN,NaN,NaN,Vietnam,Main Vocalist,Male
8,9.0,Josh,Zenith Media Contents,Z-Boys,22.0,174.0,NaN,NaN,Philippines,"Main Dancer, Lead Vocalist",Male
9,10.0,Sid,Zenith Media Contents,Z-Boys,20.0,NaN,NaN,NaN,India,Main Vocalist,Male


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1902 entries, 0 to 1901
Data columns (total 11 columns):
rank          1901 non-null float64
name          1901 non-null object
agency        1874 non-null object
group         1901 non-null object
age           1765 non-null float64
height        1603 non-null float64
weight        1253 non-null float64
blood type    1349 non-null object
country       1895 non-null object
position      1836 non-null object
gender        1901 non-null object
dtypes: float64(4), object(7)
memory usage: 163.5+ KB


In [26]:
data.head(226)

,rank,name,agency,group,age,height,weight,blood type,country,position,gender
0,1.0,Jinah,주식회사 씨그널엔터테인먼트그룹,Berry Chu,37.0,NaN,NaN,NaN,South Korea,"Leader, Vocalist",Female
1,2.0,Hayul,주식회사 씨그널엔터테인먼트그룹,Berry Chu,33.0,NaN,NaN,NaN,South Korea,Vocalist,Female
2,3.0,Jieun,주식회사 씨그널엔터테인먼트그룹,Berry Chu,33.0,NaN,NaN,NaN,South Korea,Vocalist,Female
3,4.0,Yunmi,주식회사 씨그널엔터테인먼트그룹,Berry Chu,32.0,NaN,NaN,NaN,South Korea,Vocalist,Female
4,5.0,Haum,주식회사 씨그널엔터테인먼트그룹,Berry Chu,29.0,NaN,NaN,NaN,South Korea,"Vocalist, Maknae",Female
5,6.0,Soyou,주식회사 씨그널엔터테인먼트그룹,Berry Chu,34.0,NaN,NaN,NaN,South Korea,Vocalist,Female
6,7.0,Vanya,Zenith Media Contents,Z-Girls,23.0,159.0,50.0,O,Indonesia,"Lead Vocalist, Main Dancer, Lead Rapper",Female
7,8.0,Roy,Zenith Media Contents,Z-Boys,23.0,NaN,NaN,NaN,Vietnam,Main Vocalist,Male
8,9.0,Josh,Zenith Media Contents,Z-Boys,22.0,174.0,NaN,NaN,Philippines,"Main Dancer, Lead Vocalist",Male
9,10.0,Sid,Zenith Media Contents,Z-Boys,20.0,NaN,NaN,NaN,India,Main Vocalist,Male


In [38]:
data["age"].fillna(data.groupby("name")["age"].transform("median"), inplace=True)

In [42]:
data["age"].transform("median")

ValueError: transforms cannot produce aggregated results